# Install and Setup Kubeflow on EKS

Based on these instructions:  https://www.kubeflow.org/docs/aws/deploy/install-kubeflow/

# _This Will Take a Few Minutes, Please Be Patient._

In [ ]:
%%bash

source ~/.bash_profile

export AWS_CLUSTER_STATUS=$(aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.status" --output text)

if [ -n "$AWS_CLUSTER_STATUS" ]; then
    if [ $AWS_CLUSTER_STATUS == "ACTIVE" ]; then
        echo "Cluster status:  $AWS_CLUSTER_STATUS."
    else
        echo "Cluster status:  $AWS_CLUSTER_STATUS.  Please wait for status:  ACTIVE"
        exit
    fi
fi

# DO NOT CONTINUE UNTIL THE CLUSTER STATUS IS `ACTIVE` ^^ ABOVE ^^.

In [ ]:
%%bash

source ~/.bash_profile

echo $S3_BUCKET
echo $AWS_CLUSTER_NAME
echo $INSTANCE_ROLE_NAME
echo $INSTANCE_PROFILE_ARN

sudo cp kfext/kfctl /usr/local/bin

In [ ]:
%%bash

source ~/.bash_profile

export KF_NAME=${AWS_CLUSTER_NAME}
echo "export KF_NAME=${KF_NAME}" | tee -a ~/.bash_profile

export KF_DIR=$PWD/${KF_NAME}
echo "export KF_DIR=${KF_DIR}" | tee -a ~/.bash_profile

mkdir -p ${KF_DIR}

In [ ]:
%%bash

source ~/.bash_profile

cp kfext/v1.0.2.tar.gz ${KF_DIR}
cp kfext/kfctl_aws.v1.0.2.yaml ${KF_DIR}

In [ ]:
%%bash

source ~/.bash_profile

export CONFIG_FILE=${KF_DIR}/kfctl_aws.v1.0.2.yaml
echo "export CONFIG_FILE=${CONFIG_FILE}" | tee -a ~/.bash_profile

In [ ]:
%%bash

source ~/.bash_profile

sed -i.bak -e "/region: us-west-2/ a \      enablePodIamPolicy: true" ${CONFIG_FILE}
sed -i.bak -e "s@us-west-2@$AWS_REGION@" ${CONFIG_FILE}
sed -i.bak -e "s@roles:@#roles:@" ${CONFIG_FILE}
sed -i.bak -e "s@- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@#- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@" ${CONFIG_FILE}
sed -i.bak -e "s@eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@$INSTANCE_ROLE_NAME@" ${CONFIG_FILE}
sed -i.bak -e 's/kubeflow-aws/'"$AWS_CLUSTER_NAME"'/' ${CONFIG_FILE}

In [ ]:
%%bash

source ~/.bash_profile

cd ${KF_DIR}

rm -rf kustomize
rm -rf .cache

kfctl build -V -f ${CONFIG_FILE}

cd ${KF_DIR}

kfctl apply -V -f ${CONFIG_FILE}

kubectl delete all -l app=spartakus --namespace=kubeflow

kubectl config set-context --current --namespace=kubeflow

## Wait for resource to become available
Monitor changes by running kubectl get all namespaces command.


In [ ]:
!kubectl -n kubeflow get all

In [ ]:
!kubectl -n istio-system get all

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();